# **Phase 1 - Analysing GoEmotions and VAD Mapping**

## GoEmotions DataSet

*   [GoEmotions @ Google's github](https://github.com/google-research/google-research/tree/master/goemotions/data)




In [1]:
# Set working directory as the project's dir
from google.colab import drive
drive.mount('/content/drive/')

# Change here to the path in Google Drive where this projcet is located
%cd "drive"/"My Drive"/"University"/"Year 3"/"Semester A"/"Natural Language Processing"/"Project"/"emotion-recognition-nlp-project"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/University/Year 3/Semester A/Natural Language Processing/Project/emotion-recognition-nlp-project


In [3]:
# Packages
!pip3 install datasets transformers -q

     |████████████████████████████████| 306 kB 4.1 MB/s 
     |████████████████████████████████| 3.4 MB 42.8 MB/s 
     |████████████████████████████████| 132 kB 60.2 MB/s 
     |████████████████████████████████| 1.1 MB 61.3 MB/s 
     |████████████████████████████████| 61 kB 459 kB/s 
     |████████████████████████████████| 243 kB 47.0 MB/s 
     |████████████████████████████████| 3.3 MB 35.9 MB/s 
     |████████████████████████████████| 895 kB 49.3 MB/s 
     |████████████████████████████████| 596 kB 62.1 MB/s 
     |████████████████████████████████| 192 kB 61.6 MB/s 
     |████████████████████████████████| 271 kB 37.3 MB/s 
     |████████████████████████████████| 160 kB 58.4 MB/s 


In [13]:
# Imports
import pandas as pd

In [4]:
# Load split GoEmotions to pandas DFs
from datasets import load_dataset
go_emotions = load_dataset("go_emotions", "simplified")
data = go_emotions.data
train, dev, test = data["train"].to_pandas(), data["validation"].to_pandas(), data["test"].to_pandas()

Downloading:   0%|          | 0.00/2.02k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.67k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/203k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/201k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset go_emotions downloaded and prepared to /root/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/2637cfdd4e64d30249c3ed2150fa2b9d279766bfcd6a809b9f085c61a90d776d. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
# Adding verbal emotions column

# TODO: fetch the following list from data/emotions.txt
labels_list = [
  'admiration',
  'amusement',
  'anger',
  'annoyance',
  'approval',
  'caring',
  'confusion',
  'curiosity',
  'desire',
  'disappointment',
  'disapproval',
  'disgust',
  'embarrassment',
  'excitement',
  'fear',
  'gratitude',
  'grief',
  'joya',
  'love',
  'nervousness',
  'optimism',
  'pride',
  'realization',
  'relief',
  'remorse',
  'sadness',
  'surprise',
  'neutral'
]
idx2emotion = {i : e for i, e in enumerate(labels_list)}
emotion2idx = None # TODO reverse the dict

train["emotions"] = train["labels"].apply(lambda labels: [idx2emotion[i] for i in labels])
dev["emotions"] = dev["labels"].apply(lambda labels: [idx2emotion[i] for i in labels])
test["emotions"] = test["labels"].apply(lambda labels: [idx2emotion[i] for i in labels])


## Delete multi-labels from datasets

We shall temporarily ignore multi-labaled exmaple just for simplicity. Later we will choose a way to treat them

There might be another way of doing it, as @Shir said.

In [7]:
# Add label counts column
train['labels_count'] = train['labels'].apply(lambda labels: len(labels))
dev['labels_count'] = dev['labels'].apply(lambda labels: len(labels))
test['labels_count'] = test['labels'].apply(lambda labels: len(labels))

# Print percenrage of multi-labels (candidates to delete)
print("--- What is the part that will be delete? --")
print("train : ", sum(train['labels_count'] > 1) / len(train['labels_count']))
print("dev : ", sum(dev['labels_count'] > 1) / len(dev['labels_count']))
print("test : ", sum(test['labels_count'] > 1) / len(test['labels_count']))

--- What is the part that will be delete? --
train :  0.16360285648468095
dev :  0.1618134906008109
test :  0.15422885572139303


In [8]:
train_one_label = train[train['labels_count'] == 1]
dev_one_label = dev[dev['labels_count'] == 1]
test_one_label = test[test['labels_count'] == 1]

## Map to VAD
We'd like to map each example to VAD, using its label (= emotion).
Each lexicon's df will have the column `word`, `v`, `a`, `d`

1. **First option** - [NRC-VAD : 2018](http://saifmohammad.com/WebPages/nrc-vad.html)
Crowd sourced lexicon, mapping 20K words to VAD.

In [46]:
NRC_VAD_LEXICON_PATH = "data/mapping-emotions-to-vad/nrc-vad/NRC-VAD-Lexicon.txt"
df_nrc = pd.read_csv(NRC_VAD_LEXICON_PATH, sep="\t", 
                     names=['word', 'v', 'a', 'd'])

# desired mapping from NRC
mapping_nrc = df_nrc[df_nrc["word"].apply(lambda word: word in labels_list)]
mapping_nrc.head()

,word,v,a,d
251,admiration,0.969,0.583,0.726
624,amusement,0.929,0.837,0.803
670,anger,0.167,0.865,0.657
709,annoyance,0.167,0.718,0.342
865,approval,0.854,0.460,0.889


2. **Second option** - [ANEW : 1999](https://pdodds.w3.uvm.edu/teaching/courses/2009-08UVM-300/docs/others/everything/bradley1999a.pdf)
Psychology students sourced lexicon, mapping 1K words to VAD.

In [57]:
ANEW_LEXICON_PATH = "data/mapping-emotions-to-vad/anew/all.csv"
df_anew_raw = pd.read_csv(ANEW_LEXICON_PATH, sep=",")

# mapping to conventioned dateframe
_mapper = {
    "Description": "word",
    "Valence Mean": "v",
    "Arousal Mean": "a",
    "Dominance Mean": "d",
}
_scaler = lambda x: (x - 1) / 8  # linear scaling:  [1,9] --> [0,1] # @Shir: do you think of something better?

df_anew = df_anew_raw[_mapper.keys()].rename(_mapper, axis=1)
df_anew[["v", "a", "d"]] = df_anew[["v", "a", "d"]].apply(_scaler)

# desired mapping from ANEW
mapping_anew = df_anew[df_anew["word"].apply(lambda word: word in labels_list)]
mapping_anew.head()

,word,v,a,d
172,love,0.96500,0.68000,0.76375
308,optimism,0.74375,0.54250,0.70125
360,pride,0.75000,0.60375,0.75750
714,anger,0.16750,0.82875,0.56250
884,desire,0.83625,0.79375,0.68625


In [63]:
# TODO: change dataframes indexes of mapping_anew, mapping_nrc to GoEmotions indices (like the inddexes in labels_list)